In [1]:
import urllib, requests
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook, tnrange
import random

In [13]:
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"} 
toc_url = 'https://data.tallahassee.com/fbi-cra-documents/'
response = requests.get(url=toc_url,headers=headers)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, features='lxml')


In [14]:
r = soup.find_all('a',attrs={'class':"list-group-item"})
url_paths = []
titles = []
for i in r:
    url_paths.append(i['href'])
    titles.append(i.h4.text)
# should exclude last two
num = len(url_paths)-2

In [15]:
titles = titles[:-2]
base_url = 'https://data.tallahassee.com'
urls = []
for i in range(num):
    urls.append(base_url+url_paths[i])

In [16]:
df = pd.DataFrame(data=titles, columns=['titles'])
df['urls'] = urls
df

,titles,urls
0,1: City Commission Minutes,https://data.tallahassee.com/fbi-cra-documents...
1,2: City Commission Agenda Items,https://data.tallahassee.com/fbi-cra-documents...
2,3.1: AP Payments Combined,https://data.tallahassee.com/fbi-cra-documents...
3,3.2: AR Paid COT Invoices,https://data.tallahassee.com/fbi-cra-documents...
4,3.3: PCard Charges Combined,https://data.tallahassee.com/fbi-cra-documents...
...,...,...
144,(To Be Posted) 34.4: Scott Maddox 4 (Redacted ...,https://data.tallahassee.com/fbi-cra-documents...
145,NEW 34.5: Scott Maddox 5,https://data.tallahassee.com/fbi-cra-documents...
146,NEW 34.5: Scott Maddox 5 (Redacted Files),https://data.tallahassee.com/fbi-cra-documents...
147,NEW 34.6: Scott Maddox 6,https://data.tallahassee.com/fbi-cra-documents...


In [20]:
re_findjs = re.compile('\"(http.*\.js)\"')
re_findsource = re.compile('\"resources\"\:(\{.*\})')
re_findkeyvalue = re.compile('\"(.*?)\"\:\"(.*?)\"')

meta_dicts = []
for url in tqdm_notebook(urls[-1:]):
    response = requests.get(url=url,headers=headers,timeout=5)
    response.encoding = 'utf-8'
    try:
        soup1 = BeautifulSoup(response.text, features='lxml')
        target_line1 = soup1.find('div', attrs={'class':"panel-body"}).script.text
        print("target_line1: ", target_line1)
    except:
        print(url+'not working')
        continue  
    try:
        js = re_findjs.findall(target_line1)[0]
    except:
        continue
    response = requests.get(url=js,headers=headers)
    response.encoding = 'utf-8'
    soup2 = BeautifulSoup(response.text, features='lxml')
    target_line2 = soup2.find('p').text
    dic = re_findsource.findall(target_line2)[0]
    lst = re_findkeyvalue.findall(dic)
    meta_dict = {}
    for l in lst[:7]:
        meta_dict[l[0]] = l[1]
    meta_dicts.append(meta_dict)
    url = meta_dict['pdf']
    response = requests.get(url, allow_redirects=True)
    filename = url.split('/')[-1]
    with open('./pdf/'+filename, 'wb') as f:
        f.write(response.content)
    url = meta_dict['text']
    response = requests.get(url, allow_redirects=True)
    filename = url.split('/')[-1]
    with open('./txt/'+filename, 'wb') as f:
        f.write(response.content)

<ipython-input-20-c0dc18dcca4a>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for url in tqdm_notebook(urls[-1:]):


  0%|          | 0/1 [00:00<?, ?it/s]

target_line1:  
